# Distal Results

Evaluation against distal adversarial examples.

Check `experiments/misc.py` for details on the used methods.

Make sure to set the paths and GPU id below:

In [ ]:
import os
import sys
sys.path.insert(1, os.path.abspath('') + '/../../')
from experiments.eval import misc
import torch
torch.cuda.set_device(0)
from IPython.display import display, Markdown

In [ ]:
config_module = 'config.cifar10'
config_training_variables = [
    'normal_training_check',
]
if config_module == 'config.svhn':
    config_training_variables += [
        # For MNIST:
        'adversarial_training_lr005_i40_half_momentum_backtrack_check',
        'confidence_calibrated_adversarial_training_lr001_ce_f7p_i40_random_momentum_backtrack_power2_10',
        'multi_steepest_descent',
    ]
elif config_module == 'config.svhn':
    config_training_variables += [
        # For SVHN:
        'adversarial_training_lr001_i40_half_momentum_backtrack_check',
        'confidence_calibrated_adversarial_training_ce_f7p_i40_random_momentum_backtrack_power2_10',
    ]
elif config_module == 'config.cifar10':
    config_training_variables += [
        # For Cifar10:
        'adversarial_training_lr0005_i40_half_momentum_backtrack_check',
        'confidence_calibrated_adversarial_training_ce_f7p_i40_random_momentum_backtrack_power2_10',
        'multi_steepest_descent',
    ]
else:
    assert False
config_attack_variables = [
    'acet_testing_pgd_5_f0_max_log_0001_momentum_backtrack',
    'acet_testing_smoothed_pgd_5_f0_max_log_0001_momentum_backtrack',
    'acet_testing_pgd_50_f0_max_log_0001_momentum_backtrack_10',
    'acet_testing_smoothed_pgd_50_f0_max_log_0001_momentum_backtrack_10',
]

In [ ]:
config, training_configs, attack_configs = misc.module(config_module, config_training_variables, config_attack_variables)

In [ ]:
config_training_names = [
    '\\Normal',
    '\\AdvTrain',
    '\\ConfTrain',
    '\\Wong',
]
main_config_attack_groups = [
    list(range(4)),
]
main_config_attack_names = [
    '\\begin{tabular}{@{}c@{}}dist.\\\\\\vphantom{t}\\end{tabular}',
]
supp_config_attack_names = [
    '\\Distal',
]

In [ ]:
model_files, model_epochs, perturbations_files, perturbations_epochs, new_config_training_variables, new_config_training_names = misc.load(training_configs, config_training_names, attack_configs)

In [ ]:
display(Markdown(misc.epoch_table(model_epochs, perturbations_files, perturbations_epochs, config_training_names, attack_configs)))

In [ ]:
clean_probabilities = misc.compute_clean_probabilities(model_files, config.testloader)

In [ ]:
clean_evaluations = misc.compute_clean_evaluations(config, clean_probabilities)

In [ ]:
adversarial_probabilities = misc.load_adversarial_probabilities(perturbations_files)
assert len(adversarial_probabilities) == len(clean_probabilities)

In [ ]:
main_evaluations = misc.compute_adversarial_evaluations(config, perturbations_files, adversarial_probabilities, clean_probabilities, config_training_variables, config_attack_variables, main_config_attack_groups)

In [ ]:
display(Markdown(misc.distal_markdown_table(main_evaluations, config_training_names, main_config_attack_names, tpr='99')))

In [ ]:
misc.distal_main_latex_table(config, config_training_names, main_config_attack_names, main_evaluations, tpr='98')

In [ ]:
misc.distal_main_latex_table(config, config_training_names, main_config_attack_names, main_evaluations, tpr='99')

In [ ]:
misc.distal_supp_latex_table(config, config_training_names, supp_config_attack_names, main_evaluations, tpr='99')